In [40]:
import pandas as pd

In [41]:
df_flow = pd.read_csv('images_done/attention_flow.txt', header = None)
df_rollout = pd.read_csv('images_done/attention_rollout.txt', header = None)
df_grad_rollout = pd.read_csv('images_done/attention_grad_rollout.txt', header = None)

df_blank_out = pd.read_csv('blank_out.txt', header = None)

In [42]:
df_flow.head()

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,0.091,0.086,0.087,0.088,1.000,0.086,0.089,0.086,0.086,0.088,...,0.087,0.090,0.452,0.091,0.090,0.083,0.090,0.091,0.089,0.088
1,0.737,0.076,0.079,0.074,0.093,0.088,0.085,0.082,0.084,0.083,...,0.087,0.077,0.084,0.086,0.083,0.076,0.318,0.076,0.385,0.747
2,1.000,0.031,0.030,0.030,0.030,0.030,0.031,0.030,0.030,0.030,...,0.030,0.030,0.030,0.030,0.030,0.030,0.030,0.031,0.030,0.030
3,0.066,0.066,0.067,0.066,0.065,0.064,0.065,0.066,0.065,0.074,...,0.135,0.197,0.066,0.067,0.062,0.121,0.063,0.062,0.065,0.070
4,0.038,0.037,0.037,0.037,0.036,0.037,0.037,0.037,0.036,0.036,...,0.046,0.037,0.038,0.037,0.038,0.038,0.038,0.041,0.038,0.038


In [45]:
performance_dict = {}

for idx in range(df_flow.shape[0]):
    rollout_corr = df_blank_out.iloc[idx,:].corr(df_rollout.iloc[idx,:], method='spearman')
    grad_rollout_corr = df_blank_out.iloc[idx,:].corr(df_grad_rollout.iloc[idx,:], method='spearman')
    flow_corr = df_blank_out.iloc[idx,:].corr(df_flow.iloc[idx,:], method='spearman')
    
    performance_dict[idx] = [rollout_corr,grad_rollout_corr , flow_corr]
    
df_corrs = pd.DataFrame(performance_dict).T.rename({0 : 'rollout', 1: 'grad_rollout', 2: 'flow'}, axis = 1)
df_corrs.describe()

,rollout,grad_rollout,flow
count,117.000000,117.000000,117.000000
mean,0.104985,0.127543,-0.010813
std,0.244114,0.227615,0.129585
min,-0.572575,-0.425072,-0.447661
25%,-0.054101,-0.035668,-0.088132
50%,0.072467,0.113380,-0.010015
75%,0.294970,0.292691,0.072419
max,0.650453,0.602786,0.335088


In [38]:
performance_dict = {}

for idx in range(df_flow.shape[0]):
    grad_rollout_corr = df_rollout.iloc[idx,:].corr(df_grad_rollout.iloc[idx,:], method='spearman')
    flow_corr = df_rollout.iloc[idx,:].corr(df_flow.iloc[idx,:], method='spearman')
    
    performance_dict[idx] = [grad_rollout_corr , flow_corr]
    
df_corrs = pd.DataFrame(performance_dict).T.rename({0 : 'grad_rollout', 1: 'flow'}, axis = 1)
df_corrs.describe()

,grad_rollout,flow
count,117.000000,117.000000
mean,0.477028,0.238081
std,0.207137,0.166362
min,-0.036626,-0.203054
25%,0.330677,0.136052
50%,0.520877,0.239694
75%,0.634525,0.337783
max,0.835514,0.661277


In [48]:
performance_dict = {}

for idx in range(df_flow.shape[0]):
    rollout_corr = df_flow.iloc[idx,:].corr(df_rollout.iloc[idx,:], method='spearman')
    grad_rollout_corr = df_flow.iloc[idx,:].corr(df_grad_rollout.iloc[idx,:], method='spearman')
    
    performance_dict[idx] = [rollout_corr , grad_rollout_corr]
    
df_corrs = pd.DataFrame(performance_dict).T.rename({0 : 'rollout', 1: 'grad'}, axis = 1)
df_corrs.describe()

,rollout,grad
count,117.000000,117.000000
mean,0.238081,0.205036
std,0.166362,0.214552
min,-0.203054,-0.475615
25%,0.136052,0.050187
50%,0.239694,0.216467
75%,0.337783,0.352497
max,0.661277,0.757209


/Data/nlp_env/miniconda3/envs/venv_ea/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/Data/nlp_env/miniconda3/envs/venv_ea/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


NameError: name 'Image' is not defined